In [3]:
name = "up_blocks.1.resnets.2.temporal_res_block.conv1.bias"

print("yes" if "up_blocks" in name else "no")

yes


In [8]:
from datasets import Dataset, DatasetDict, Image
import os
import json

data_dict = {
    'scene_name': [],
    'caption': [],
    'conditioning_images_one': [],
    'ground_truth': []
}

# This function can be used for edges rgb seg
def get_image_paths_prescan(dir, indexes, type):
    image_paths = []
    out_of_range = False
    for index in indexes:

        file_name = "frame"
        index = str(index)
        try:
            image_paths.append(os.path.join(dir, f"{file_name}{index}.png"))
        except:
            out_of_range = True
            break
    

    image_paths.sort(key=lambda x: int(x.split("/")[-1].split('frame')[1].split('.')[0]))
    
    
    return image_paths, out_of_range


def get_captions_prescan(dir):
    caption = []
    with open(dir, 'r') as file:
        data = file.read()
        caption.append(data)
    return caption

def calculate_the_indexes_of_prescan(len_frames_ground_truth, len_frames_prescan, indexes):
    # Correct the use of max_frames and min_frames
    mapped_indexes = []
    min_frames = min(len_frames_ground_truth, len_frames_prescan)
    max_frames = max(len_frames_ground_truth, len_frames_prescan)

    for frame_idx in indexes:
        # Correct the calculation of current_frame
        current_frame = min(int(frame_idx * (max_frames / min_frames)), max_frames - 1)
        mapped_indexes.append(current_frame)

    return mapped_indexes

    

prescan_root_dir = "/mnt/e/13_Jasper_diffused_samples/complete_dataset/prescan_normal_output/new_scenes_for_new_model"
nuscene_root_dit = "/mnt/e/13_Jasper_diffused_samples/nuscene_frames"
scene_names = os.listdir(prescan_root_dir)

# pop all the dirs with seg in the name
scene_names = [scene for scene in scene_names if "seg" not in scene]

# Create an error.txt file
error_file = open("error.txt", "w")


for scene in scene_names:

    try:
        # First we need to try and get the paths to all the folders and check if there are files in them
        # If there are no files in the folder, we need to skip it

        ground_truth_path = os.path.join(nuscene_root_dit, scene) 
        caption_path = os.path.join(prescan_root_dir, scene, "caption") # should contain one file
        conditioning_images_one_path = os.path.join(nuscene_root_dit, scene + "_seg")

        # in the reference_image, conditioning_images_one_path, conditioning_images_two_path, we need to check if there are files in the folder and the same amount
        # get the files in the folders
        cond_files_one = len(os.listdir(conditioning_images_one_path))
        cap_files = len(os.listdir(caption_path))
        ground_truth_files = len(os.listdir(ground_truth_path))


        # We need to take bites of 25 frames from the ground_truth folder

        number_of_segments = len(os.listdir(ground_truth_path)) // 14 

        # Now we need to get the corresponding images for each segment
        for i in range(number_of_segments):
            offset = 5
            indexes_nuscenes = range(i * 14 + offset, (i + 1) * 14 + offset)
            ground_truth_indexes = range(i * 14 , (i + 1) * 14 )
            indexes_prescan = calculate_the_indexes_of_prescan(ground_truth_files, cap_files, indexes_nuscenes)                 # add the indexes that need to be calculated
            index_reference_image = [indexes_prescan[0]]

            # get the images path
            data_ground_truth, out_of_range1 = get_image_paths_prescan(ground_truth_path, ground_truth_indexes, "ground_truth")
            data_conditioning_images_one, out_of_range3 = get_image_paths_prescan(conditioning_images_one_path, ground_truth_indexes, "seg")
            data_caption = get_captions_prescan(caption_path + "/" + "caption.txt")


            

            # Now all the values need to appended to the dictornary
            data_dict['scene_name'].append(scene)
            data_dict['ground_truth'].append(data_ground_truth)
            data_dict['caption'].append(data_caption)
            data_dict['conditioning_images_one'].append(data_conditioning_images_one)


            # stop the loop
        #     if i == 2:
        #         break 
        # break

    except Exception as e:  
        # Write to the error file
        print(e)
        error_file.write(f"{scene} caused an error: {e}\n")




# Save the data_dict to a json file
with open('jappie_seg.json', 'w') as file:
    json.dump(data_dict, file, indent=4)



[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0103_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0361_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0378_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0401_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0420_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0421_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0426_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0449_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0535_seg'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/n